In [99]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
import tensorflow
import requests
from bs4 import BeautifulSoup

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
response=requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')
soup=BeautifulSoup(response.content,'html.parser')


## Data Cleaning

In [3]:
import spacy
import re,string
# from nltk.corpus import stopwords
# nlp = spacy.load('en_core_web_sm')
# nlp.max_length = 5281520

# def preprocess(texts):
#     doc = nlp(texts)
#     text=" " .join([token.lemma_ for token in doc])
#     text = re.sub(r'\&\w*;','',text)
#     text = re.sub('@[^\s]+','',text)   
#     text = re.sub(r'https?:\/\/.*\/\w*','',text)
#     text = re.sub(r'#\w*','',text)
#     text = re.sub(r'\s\s+',' ',text)
#     text = ''.join(c for c in text if c <= '\uFFFF')
#     return text
def preprocess(file):
  tokens = file.split()#Split by Whitespace
  table = str.maketrans(' ', ' ', string.punctuation)#We can use the function maketrans() to create a mapping table
  tokens = [w.translate(table) for w in tokens]#Python offers a function called translate() that will map one set of characters to another.
  tokens = [word for word in tokens if word.isalpha()]#The isalpha() method returns True if all the characters are alphabet letters (a-z).
  return tokens



In [49]:
print(preprocess("In the text preprocessing function, the input text is first tokenized and lemmatized using spaCy to obtain a list of lemmatized tokens. This is followed by the removal of HTML entities, mentions (e.g., @username), URLs, and hashtags using regular expressions. Consecutive whitespaces are then reduced to a single space, and non-Unicode characters are removed, ensuring that the resulting text is clean and standardized. This comprehensive preprocessing pipeline aims to enhance the quality of the text data for downstream natural language processing tasks, such as sentiment analysis or text generation. Adjustments to the preprocessing steps can be made based on the specific requirements of the task at hand."))

['In', 'the', 'text', 'preprocessing', 'function', 'the', 'input', 'text', 'is', 'first', 'tokenized', 'and', 'lemmatized', 'using', 'spaCy', 'to', 'obtain', 'a', 'list', 'of', 'lemmatized', 'tokens', 'This', 'is', 'followed', 'by', 'the', 'removal', 'of', 'HTML', 'entities', 'mentions', 'eg', 'username', 'URLs', 'and', 'hashtags', 'using', 'regular', 'expressions', 'Consecutive', 'whitespaces', 'are', 'then', 'reduced', 'to', 'a', 'single', 'space', 'and', 'nonUnicode', 'characters', 'are', 'removed', 'ensuring', 'that', 'the', 'resulting', 'text', 'is', 'clean', 'and', 'standardized', 'This', 'comprehensive', 'preprocessing', 'pipeline', 'aims', 'to', 'enhance', 'the', 'quality', 'of', 'the', 'text', 'data', 'for', 'downstream', 'natural', 'language', 'processing', 'tasks', 'such', 'as', 'sentiment', 'analysis', 'or', 'text', 'generation', 'Adjustments', 'to', 'the', 'preprocessing', 'steps', 'can', 'be', 'made', 'based', 'on', 'the', 'specific', 'requirements', 'of', 'the', 'task', 

In [4]:
data=preprocess(soup.get_text())

In [59]:
print(len(data))

871786


In [61]:
len(set(data))#total

33982

In [5]:
#here we predict the word 51th postion using 50th position
#divide our data in chunks of 51 words and at the last we will separate the last word from every line
length_sentence = 50 + 1
lines = []
for i in range(length_sentence, len(data)):
  seq = data[i-length_sentence:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 100000:#limit our dataset to 200000 words.
    break

print(len(lines))

99951


In [69]:
lines[0]

'This is the Etext file presented by Project Gutenberg and is presented in cooperation with World Library Inc from their Library of the Future and Shakespeare CDROMS Project Gutenberg often releases Etexts that are NOT placed in the Public Domain Shakespeare This Etext has certain copyright implications you should read Project'

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

tokenizer=Tokenizer()
tokenizer.fit_on_texts(lines)
sequence=tokenizer.texts_to_sequences(lines)



In [7]:
max_classes=len(tokenizer.word_index)+1


In [8]:
sequences= np.array(sequence)
x = sequences[:, :-1]
yx = sequences[:, -1]

y=to_categorical(yx,num_classes=max_classes)

In [42]:
import numpy as np

# Assuming yx is your array of indices
unique_values = np.unique(yx)

# Check if the minimum value is 0 or 1
if np.min(unique_values) == 0:
    print("Indices are zero-based.")
else:
    print("Indices are one-based.")


Indices are zero-based.


In [9]:
seq_length = x.shape[1]
seq_length

50

In [10]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size


yx

array([616, 912,  11, ...,  12, 109,   5])

In [142]:
import numpy as np

# Example sequences
sequences = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Separate input sequences (x) and target sequences (y)
x = sequences[:, :-1]
y = sequences[:, -1]

print("Input Sequences (x):")
print(x)
print("\nTarget Sequences (y):")
print(y)

Input Sequences (x):
[[1 2]
 [4 5]
 [7 8]]

Target Sequences (y):
[3 6 9]


In [16]:
print("Shape of x:", x.shape)
seq_length= x.shape[1]
print(seq_length)

Shape of x: (199951, 50)
50


In [17]:
from keras.layers import LSTM,Dense,Embedding,Dropout
from keras.models import Sequential
model=Sequential()
model.add(Embedding(max_classes,60,input_length=seq_length)) 
model.add(LSTM(units=32,return_sequences=True))
model.add(LSTM(units=16))




model.add(Dense(16,activation='relu'))
model.add(Dense(max_classes, activation="softmax"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 50, 60)            548220    
                                                                 
 lstm_6 (LSTM)               (None, 50, 32)            11904     
                                                                 
 lstm_7 (LSTM)               (None, 16)                3136      
                                                                 
 dense_8 (Dense)             (None, 16)                272       
                                                                 
 dense_9 (Dense)             (None, 9137)              155329    
                                                                 
Total params: 718861 (2.74 MB)
Trainable params: 718861 (2.74 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
from keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
model.fit(
    x,
    y,
    batch_size=32,
    epochs=20,
    verbose=1
)

Epoch 1/20
3124/3124 [==============================] - 65s 20ms/step - loss: 6.8524 - accuracy: 0.0288
Epoch 2/20
3124/3124 [==============================] - 41s 13ms/step - loss: 6.5368 - accuracy: 0.0367
Epoch 3/20
3124/3124 [==============================] - 40s 13ms/step - loss: 6.3637 - accuracy: 0.0459
Epoch 4/20
3124/3124 [==============================] - 39s 12ms/step - loss: 6.2190 - accuracy: 0.0573
Epoch 5/20
3124/3124 [==============================] - 39s 12ms/step - loss: 6.0858 - accuracy: 0.0638
Epoch 6/20
3124/3124 [==============================] - 39s 12ms/step - loss: 5.9652 - accuracy: 0.0689
Epoch 7/20
3124/3124 [==============================] - 39s 12ms/step - loss: 5.8626 - accuracy: 0.0772
Epoch 8/20
3124/3124 [==============================] - 39s 12ms/step - loss: 5.7739 - accuracy: 0.0833
Epoch 9/20
3124/3124 [==============================] - 39s 13ms/step - loss: 5.6937 - accuracy: 0.0878
Epoch 10/20
3124/3124 [==============================] - 39s 12m

In [25]:
def generate_text(seed_text, model, tokenizer, max_sequence_length, num_words_to_generate):
    generated_text = seed_text  # Initialize with the seed text
    for _ in range(num_words_to_generate):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        while len(token_list) < max_sequence_length - 1:
            token_list.insert(0, 0)  
        token_list = token_list[-(max_sequence_length-1):]
        input_sequence = np.array(token_list)
        input_sequence = np.reshape(input_sequence, (1, -1))
        predicted_probs = model.predict(input_sequence, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_word = tokenizer.index_word.get(predicted_index, '')
        seed_text += ' ' + predicted_word
        generated_text += ' ' + predicted_word

    return generated_text

# Example Usage
generated_text = generate_text(seed_text='''The''', model=model, tokenizer=tokenizer,
                               max_sequence_length=seq_length+1, num_words_to_generate=100)
print("Generated Text:", generated_text)
print("Generated Text:", len(generated_text))



Generated Text: The world and be done and we have done and we have been done and thou hast not be done and i have done and i have been done and i have been done and i have been done and i have been done and i have been done and i have been not not and be done and i have been done and i have been done and i have been not not and be done and i have been done and i have been not not and be done and i have been done and i have been not
Generated Text: 419


In [20]:
def generate_text(seed_text, model, tokenizer, max_sequence_length, num_words_to_generate):
    generated_text = seed_text  # Initialize with the seed text
    for _ in range(num_words_to_generate):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        while len(token_list) < max_sequence_length - 1:
            token_list.insert(0, 0)  
        token_list = token_list[-(max_sequence_length-1):]
        input_sequence = np.array(token_list)
        input_sequence = np.reshape(input_sequence, (1, -1))
        predicted_probs = model.predict(input_sequence, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_word = tokenizer.index_word.get(predicted_index, '')
        seed_text += ' ' + predicted_word
        generated_text += ' ' + predicted_word

    return generated_text

# Example Usage
generated_text = generate_text(seed_text='''The Bird sang''', model=model, tokenizer=tokenizer,
                               max_sequence_length=seq_length+1, num_words_to_generate=123)
print("Generated Text:", generated_text)
print("Generated Text:", len(generated_text))



Generated Text: The Bird sang and i am not not not and be done and be done and we have been done and we have been done and thou hast not be done and i have done and i have been done and i have been done and i have been done and i have been done and i have been done and i have been not not and be done and i have been done and i have been done and i have been not not and be done and i have been done and i have been not not and be done and i have been done and i have been not not and be done and i have been done and i have been not
Generated Text: 518
